In [1]:
from manim import *

## Fade-In/Fade-Out

Note that playing an animation on an object __automatically adds it to the scene__.

In [2]:
%%manim -v WARNING -qm Fade

class Fade(Scene):
    def construct(self):
        circle = Circle()
        
        self.play(FadeIn(circle))  # NOTE: you didn't have to do self.add() because of doing the fade-in
        self.wait()
        self.play(FadeOut(circle))

## Rotation


In [17]:
%%manim -v WARNING -qm Rotation

class Rotation(Scene):
    def construct(self):
        square = Square()   # note that you didn't have to add the square first to animate it
        self.play(Rotate(square, PI * 2))  # rotate by radians (and use PI constant)

## Text Write-On

See __latex__ notebook for more about this animation since it works for `Tex()` objects too.

In [2]:
%%manim -v WARNING -qm TextWrite

class TextWrite(Scene):
    def construct(self):
        self.play(Write(Text("Hello", font_size=144)))

Manim Community v0.18.1

## Animating Characters in Text

You should be able to index characters within text and use them as shapes for transformations.
`Transform(text["H"][0], circle)`

NOTE: this is supposed to work but I can't create `Text()` objects on my system.

## Anagrams

`TransformMatchingShapes()` can transform 1 string into another with shapes (letters) that match getting moved instead of morphed/replaced.

## Mobject.animate


In [24]:
%%manim -v WARNING -qm PropertyAnimation

class PropertyAnimation(Scene):
    def construct(self):
        square = Square()
        # Animate 3 property changes at once by chaining.
        self.play(square.animate.shift(UP).rotate(PI / 2).set_fill(BLUE, opacity=1.0))
        # NOTE: the size of the square is inconsistent due to interpolation of properties.

## Animation Timing

Animations default to 1 second with easing, but you can override both of these things.

In [6]:
%%manim -v WARNING -qm AnimationRuntime

class AnimationRuntime(Scene):
    def construct(self):
        square = Square()
        self.play(FadeIn(square), run_time=3, rate_func=linear)  # 3 seconds instead of defualt 1 (and linear interpolation)

## Updater

This is useful for things like having a dot follow a curve as its drawn!

In [5]:
%%manim -v WARNING -qm Updater

class Updater(Scene):
    def construct(self):
        square = Square()
        square.add_updater(lambda x: x.move_to(ORIGIN))  # every animation frame, we reset position back to oring
        
        # Both of these property changes will happen and then the updater will run, before animation frame rendered
        self.play(square.animate.move_to(LEFT).set_fill(BLUE, opacity=1))

## Custom Counter Animation


In [2]:
%%manim -v WARNING -qm CountingScene

# Inherit from Animation class to define a custom animation.
class Count(Animation):
    # The c'tor can take whatever you need it to take, plus what the superclass needs.
    def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
        # Pass number as the mobject of the animation
        super().__init__(number,  **kwargs)
        # Set start and end
        self.start = start
        self.end = end

    # Implement this method to update the animation's current mobject based on % of animation done.
    def interpolate_mobject(self, alpha: float) -> None:
        # Set value of DecimalNumber according to alpha
        value = self.start + (alpha * (self.end - self.start))
        self.mobject.set_value(value)  # mobject wrapped by the animation
        
class CountingScene(Scene):
    def construct(self):
        # Create Decimal Number and add it to scene
        number = DecimalNumber().set_color(WHITE).scale(5)
        # Add an updater to keep the DecimalNumber centered as its value changes
        number.add_updater(lambda number: number.move_to(ORIGIN))

        self.add(number)

        self.wait()

        # Play the Count Animation to count from 0 to 100 in 4 seconds
        self.play(Count(number, 0, 100), run_time=4, rate_func=linear)

        self.wait()

## Custom Shape Following Animation


In [7]:
%%manim -v WARNING -qm ShapeFollowingScene

class ShapeFollow(Animation):
    def __init__(self, shape, **kwargs) -> None:
        self.shape = shape
        super().__init__(Dot(shape.get_start()).scale(2).set_color(BLUE),  **kwargs)
        
    def interpolate_mobject(self, alpha: float) -> None:
        self.mobject.move_to(self.shape.point_from_proportion(alpha))
        
class ShapeFollowingScene(Scene):
    def construct(self):
        self.play(ShapeFollow(Triangle()), run_time=4, rate_func=linear)

## Transforming

HINT: if you see weird vertex movements, try using np.roll() and/or flip() to improve it.


In [8]:
%%manim -v WARNING -qm TransformShapes

class TransformShapes(Scene):
    def construct(self):
        circle = Circle().set_fill(BLUE, opacity=1).scale(5)
        square = Square().set_fill(RED, opacity=1).scale(3)
        
        self.play(Transform(circle, square))

## Magnetic Object


In [14]:
%%manim -v WARNING -qm Magnetic

class Magnetic(Scene):
    def construct(self):
        triangle = Triangle()
        dot = Dot()
        self.add(triangle, dot)
        
        # You don't need to "update" the dot for its updater to get called (called per animation frame).
        dot.add_updater(lambda x: x.next_to(triangle, DOWN))
        self.play(triangle.animate.shift(RIGHT * 2))
        

## Wait

`self.wait` is just a delay (like `sleep()` in Python, but for animation).  The default duration is `1.0` seconds.

In [4]:
%%manim -v WARNING -qm Waiting

class Waiting(Scene):
    def construct(self):
        triangle = Triangle()
        square = Square()
        dot = Dot()
        self.add(triangle)
        self.wait()
        self.add(square)
        self.wait(duration=3.0)
        self.add(dot)
        self.wait()

## Timing of Animation Frames

In [8]:
%%manim -v WARNING -qm Timing

import time

class Timing(Scene):
    def construct(self):
        triangle = Triangle()
        self.add(triangle)    # nothing shown yet
        time.sleep(2)         # 2 second delay before anything shown
        triangle.move_to(LEFT*3)  # transform happens before shown, so original never seen
        self.wait()
        triangle.move_to(RIGHT*3)  # this is never seen because no animation is played

## Controlling Durations

Durations can be specified on multiple levels, but ultimately, if specified on a higher level, that __overrides the lower level__.

If multiple animations play in __parallel__ without a higher-level override, they all play for their __specified amounts__.

In [9]:
%%manim -v WARNING -qm Durations

class Durations(Scene):
    def construct(self):
        circle = Circle()
        
        # the self.play() versions are used in all these cases
        self.play(FadeIn(circle), run_time=3)
        self.play(FadeOut(circle), run_time=1)
        self.play(FadeIn(circle, run_time=5), run_time=1)
        self.play(FadeOut(circle, run_time=1), run_time=5)
        
        # parallel animations
        self.play(FadeIn(circle, run_time=1), FadeIn(Square(), run_time=5))

## Parallel Animations

Many methods, including `self.play()` and constructors for animations take __multiple positional arguments__ allowing for parallelism.

NOTE: this makes it flexible for use with collections, generators, etc.

In [12]:
%%manim -v WARNING -qm Parallel

class Parallel(Scene):
    def construct(self):
        circle = Circle()
        square = Square()
        shapes = [circle, square]
        
        self.play(FadeIn(circle), FadeIn(square))
        self.play(FadeOut(*shapes))
        self.play(*(FadeIn(circle), FadeIn(square)))

## Re-Creating

Animations that create an object, such as `FadeIn`, `Create`, and `Write` can be redone without having to do the inverse first.  The effect will be that the object will __suddenly dissappear__ to start the animation again.

__NOTE__: if no animation frames happened in between, then the sudden disappears won't even be seen.  For instance, if you add an object to the scene, then add it to a list of fixed in frame objects, then do `Create` on it.

In [3]:
%%manim -v WARNING -qm Recreating

class Recreating(Scene):
    def construct(self):
        circle = Circle()
        square = Square()
        shapes = [circle, square]
        
        self.play(FadeIn(circle), FadeIn(square), run_time=5)
        self.play(*(FadeIn(circle), FadeIn(square)), run_time=5)
        self.play(Create(circle), Create(square), run_time=5)

## Other Ways to Follow Circular Paths

Also see https://docs.manim.community/en/stable/examples.html#pointwithtrace

In [9]:
%%manim -v WARNING -qm FollowPathsOther

class FollowPathsOther(Scene):
    def construct(self):
        circle = Circle()  # not in the scene (but could be with opacity 0 as well)
        dot = Dot()
        
        self.play(MoveAlongPath(dot, circle))
        self.play(Rotating(dot, about_point=[2, 2, 0], radians=PI), run_time=1)

## Conceptual Summary of Animation

 * the `construct` method of the scene is the high-level flow
   * don't confuse it with something like an animation loop
   * no animation happens until you call a method like `self.play()` or `self.animation.*` or `self.wait()`
   * in between those calls, nothing happens in the video (no frames) - but calculations can be done
   * the resulting video is the accumulative of all those calls, no matter what happened in between
   * NOTE: the code in `construct` affects your render time, but only animations affect video timing
 * shapes are math objects with state and thus can be used independently of the animation
   * eg. can have a circle that is not added to the scene and use that as a path for other shapes to move on
   * keep in mind that if you animate property changes for them, the math object is changed
 * operations on objects change the underlying mathematical object instantaneously
   * if it's already added to the scene, it will instantly change in the scene
   * if no animation happened in between, the first state will not be seen
 * animation objects or chained animation calls change the math object in a way that is visible in the scene (with timing)
   * as opposed to just changing the properties directly, which changes the math object instantly and doesn't display yet
 * animation calls are __synchronous__
   * you can do parallel animations by passing multiple into `self.play`